In [1]:
import csv
import psycopg2
from psycopg2 import sql
from datetime import datetime
import logging
import os
from dotenv import load_dotenv

In [2]:
from google.cloud import secretmanager
def get_secret(secret_id, project_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("utf-8")

In [3]:
PROJECT_ID = "cloud-computing-project-458205"
load_dotenv()
DB_NAME = os.getenv("DB_NAME", "database-instance")
DB_USER = get_secret("DB_USER", PROJECT_ID).replace("\n", "")
DB_PASSWORD = get_secret("DB_PASSWORD", PROJECT_ID).replace("\n", "")
DB_HOST = get_secret("DB_HOST", PROJECT_ID).replace("\n", "")
DB_PORT = os.getenv("DB_PORT", "5432")

In [4]:
from psycopg2.extras import DictCursor
conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            cursor_factory=DictCursor,  # Use DictCursor to access columns by name
        )
conn

<connection object at 0x798a2c15afc0; dsn: 'user=app-engine-user password=xxx dbname=database-instance host=34.140.62.43 port=5432', closed: 0>

In [6]:
create_table_query = """
    DROP TABLE IF EXISTS fingerprints;
    CREATE TABLE fingerprints (
        fingerprint_id SERIAL PRIMARY KEY,
        song_id INTEGER NOT NULL REFERENCES tracks(track_id) ON DELETE CASCADE,
        hash TEXT NOT NULL,
        timestamp FLOAT NOT NULL,
        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP,
        UNIQUE (song_id, hash, timestamp)
    );
    CREATE INDEX idx_fingerprints_hash ON fingerprints(hash);
    """

In [28]:
with conn.cursor() as cursor:
    cursor.execute(create_table_query)
    conn.commit()
    print("Fingerprints table created successfully.")

Fingerprints table created successfully.


In [1]:
from src.abracadabra.GCPFingerprintDB import GCPFingerprintDB

db = GCPFingerprintDB()
db.show_table()

(1, 'In Da Club')
(2, 'In Da Club - Single Version / Edit')
(3, 'Just A Lil Bit')
(4, 'P.I.M.P.')
(5, 'Window Shopper')
(6, 'Back In Black')
(7, 'Dirty Deeds Done Dirt Cheap')
(8, 'Heatseeker')
(9, 'High Voltage')
(10, 'Highway to Hell')
(11, "It's a Long Way to the Top (If You Wanna Rock 'N' Roll)")
(12, 'Jailbreak')
(13, "Rock and Roll Ain't Noise Pollution")
(14, 'Shoot to Thrill')
(15, 'T.N.T.')
(16, 'Thunderstruck')
(17, 'Who Made Who')
(18, 'You Shook Me All Night Long')
(19, 'Chasing Pavements')
(20, "Don't You Remember")
(21, 'Easy On Me')
(22, "He Won't Go")
(23, 'Hello')
(24, 'Love In The Dark')
(25, 'Make You Feel My Love')
(26, 'Million Years Ago')
(27, 'One and Only')
(28, 'Remedy')
(29, 'Rolling in the Deep')
(30, 'Rumour Has It')
(31, 'Send My Love (To Your New Lover)')
(32, 'Set Fire to the Rain')
(33, 'Skyfall')
(34, 'Someone Like You')
(35, 'Sweetest Devotion')
(36, 'Take It All')
(37, 'Turning Tables')
(38, 'Water Under The Bridge')
(39, 'When We Were Young')
(40, 'A

In [2]:
print(db.load_tracks_from_db())

[(94, 'bad guy', 'https://www.youtube.com/watch?v=DyDfgMOUjCI'), (95, 'bury a friend', 'https://www.youtube.com/watch?v=HUHC9tYz8ik'), (96, 'everything i wanted', 'https://www.youtube.com/watch?v=EgBJmlPo8Xw'), (97, "when the party's over", 'https://www.youtube.com/watch?v=pbMwTqkKSps'), (98, 'wish you were gay', 'https://www.youtube.com/watch?v=yaJx0Gj_LCY'), (99, 'BIG LOVE', 'https://www.youtube.com/watch?v=rPmcLzgnlo8'), (100, 'Boom Boom Pow', 'https://www.youtube.com/watch?v=4m48GqaOz90'), (101, "Don't Lie", 'https://www.youtube.com/watch?v=WbJNkH-pDd8'), (102, "Don't Phunk With My Heart", 'https://www.youtube.com/watch?v=P4Bda6_usuc'), (103, 'Don’t Stop The Party', 'https://www.youtube.com/watch?v=u9LH_y159sg'), (104, 'Hey Mama', 'https://www.youtube.com/watch?v=rtczBseiAac'), (105, 'I Gotta Feeling', 'https://www.youtube.com/watch?v=uSD4vsh1zDA'), (106, 'Imma Be', 'https://www.youtube.com/watch?v=kdAj-dBNCi4'), (107, 'Just Can’t Get Enough', 'https://www.youtube.com/watch?v=OrTyD

In [ ]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp")

  0%|          | 0/360 [00:00<?, ?it/s]WARNING: [youtube] ITuOddPeYoc: Some tv client https formats have been skipped as they are DRM protected. The current session may have an experiment that applies DRM to all videos on the tv client. See  https://github.com/yt-dlp/yt-dlp/issues/12563  for more details.


[download]  48.9% of ~   7.01MiB at  627.04KiB/s ETA 00:13 (frag 44/92) 

  0%|          | 1/360 [01:26<8:38:27, 86.65s/it]

Error indexing Wanted Dead Or Alive (ID 136): [Errno 2] No such file or directory: '../../songs/Bon Jovi - Wanted Dead Or Alive (Official Music Video).m4a'
[download]  56.8% of ~   7.15MiB at  750.90KiB/s ETA 00:08 (frag 53/92)Error indexing Gimme More (ID 146): [Errno 2] No such file or directory: '../../songs/Britney Spears - Gimme More (Official HD Video).m4a'
[download]  58.7% of ~   6.91MiB at  768.79KiB/s ETA 00:08 (frag 53/92)

  1%|          | 2/360 [01:27<3:34:55, 36.02s/it]

...

 26%|██▋       | 95/360 [34:11<2:28:05, 33.53s/it]

 27%|██▋       | 96/360 [34:12<1:44:27, 23.74s/it]

[download]  10.2% of    4.89MiB at  983.76KiB/s ETA 00:04

 27%|██▋       | 96/360 [34:21<1:34:28, 21.47s/it]

[download]  20.4% of    4.89MiB at    1.21MiB/s ETA 00:03

In [1]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp", skip_duplicates=True)

Skipping In Da Club (ID 1) — already indexed.
Skipping In Da Club - Single Version / Edit (ID 2) — already indexed.
Skipping Just A Lil Bit (ID 3) — already indexed.
Skipping P.I.M.P. (ID 4) — already indexed.
Skipping Window Shopper (ID 5) — already indexed.
Skipping Back In Black (ID 6) — already indexed.
Skipping Dirty Deeds Done Dirt Cheap (ID 7) — already indexed.
Skipping Heatseeker (ID 8) — already indexed.
Skipping High Voltage (ID 9) — already indexed.
Skipping Highway to Hell (ID 10) — already indexed.
Skipping It's a Long Way to the Top (If You Wanna Rock 'N' Roll) (ID 11) — already indexed.
Skipping Jailbreak (ID 12) — already indexed.
Skipping Rock and Roll Ain't Noise Pollution (ID 13) — already indexed.
Skipping Shoot to Thrill (ID 14) — already indexed.
Skipping T.N.T. (ID 15) — already indexed.
Skipping Thunderstruck (ID 16) — already indexed.
Skipping Who Made Who (ID 17) — already indexed.
Skipping You Shook Me All Night Long (ID 18) — already indexed.
Skipping Chasi

  0%|          | 0/491 [00:00<?, ?it/s]

 47%|████▋     | 232/491 [00:51<00:57,  4.49it/s]

Error: The downloaded file for track 205 is missing or empty.
Error indexing Genie In a Bottle (ID 205): cannot unpack non-iterable NoneType object
  

 47%|████▋     | 233/491 [05:52<08:51,  2.06s/it]

...

ERROR: [youtube] NxJ7O0CGzO8: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
 97%|█████████▋| 474/491 [1:27:31<00:09,  1.72it/s]

[Attempt 2] Download error: ERROR: [youtube] NxJ7O0CGzO8: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Error indexing SoulMate (ID 490): cannot unpack non-iterable NoneType object


ERROR: [youtube] -rjhH3SMQ-0: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
 97%|█████████▋| 475/491 [1:27:31<00:07,  2.02it/s]

[Attempt 2] Download error: ERROR: [youtube] -rjhH3SMQ-0: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Error indexing Summer Love (ID 491): cannot unpack non-iterable NoneType object


100%|██████████| 491/491 [1:32:20<00:00, 11.28s/it]


In [5]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp", skip_duplicates=True)

Skipping In Da Club (ID 1) — already indexed.
Skipping In Da Club - Single Version / Edit (ID 2) — already indexed.
Skipping Just A Lil Bit (ID 3) — already indexed.
Skipping P.I.M.P. (ID 4) — already indexed.
Skipping Window Shopper (ID 5) — already indexed.
Skipping Back In Black (ID 6) — already indexed.
Skipping Dirty Deeds Done Dirt Cheap (ID 7) — already indexed.
Skipping Heatseeker (ID 8) — already indexed.
Skipping High Voltage (ID 9) — already indexed.
Skipping Highway to Hell (ID 10) — already indexed.
Skipping It's a Long Way to the Top (If You Wanna Rock 'N' Roll) (ID 11) — already indexed.
Skipping Jailbreak (ID 12) — already indexed.
Skipping Rock and Roll Ain't Noise Pollution (ID 13) — already indexed.
Skipping Shoot to Thrill (ID 14) — already indexed.
Skipping T.N.T. (ID 15) — already indexed.
Skipping Thunderstruck (ID 16) — already indexed.
Skipping Who Made Who (ID 17) — already indexed.
Skipping You Shook Me All Night Long (ID 18) — already indexed.
Skipping Chasi

 81%|████████  | 396/491 [03:57<00:56,  1.67it/s]

 81%|████████  | 397/491 [04:04<00:58,  1.60it/s]

 81%|████████  | 398/491 [04:09<00:59,  1.55it/s]

 81%|████████▏ | 399/491 [04:27<01:10,  1.30it/s]

...

 95%|█████████▌| 468/491 [57:57<27:50, 72.62s/it]

 96%|█████████▌| 469/491 [58:00<18:56, 51.67s/it]

 96%|█████████▌| 470/491 [58:15<14:13, 40.65s/it]

 96%|█████████▌| 471/491 [1:00:31<23:05, 69.25s/it]

100%|██████████| 491/491 [1:06:54<00:00,  8.18s/it]


In [11]:
with conn.cursor() as cur:
    cur.execute("SELECT * FROM tracks where artist_names LIKE 'PSY';")
    rows = cur.fetchall()
    for row in rows:
        print(row)

In [8]:
from src.abracadabra.recognize import recognize_song
match = recognize_song("../songs/Adele - Someone Like You (Official Music Video).m4a", db)

if match:
    print(f"Matched: {match[0]} (score: {match[1]})")
    track_name, artist_names, album_name, youtube_url = db.check_song_info(match[0])
    # track_name, youtube_url = db.check_song_info(match[0])
    print(f"Track Name: {track_name}")
    print(f"Artist Names: {artist_names}")
    print(f"Album Name: {album_name}")
    print(f"YouTube URL: {youtube_url}")
else:
    print("No match found.")

Matched: 34 (score: 25)
Track Name: Someone Like You


NameError: name 'artist_names' is not defined

In [8]:
from src.abracadabra.GCPFingerprintDB import GCPFingerprintDB
db = GCPFingerprintDB()
db.show_table()

(1, 'In Da Club')
(2, 'In Da Club - Single Version / Edit')
(3, 'Just A Lil Bit')
(4, 'P.I.M.P.')
(5, 'Window Shopper')
(6, 'Back In Black')
(7, 'Dirty Deeds Done Dirt Cheap')
(8, 'Heatseeker')
(9, 'High Voltage')
(10, 'Highway to Hell')
(11, "It's a Long Way to the Top (If You Wanna Rock 'N' Roll)")
(12, 'Jailbreak')
(13, "Rock and Roll Ain't Noise Pollution")
(14, 'Shoot to Thrill')
(15, 'T.N.T.')
(16, 'Thunderstruck')
(17, 'Who Made Who')
(18, 'You Shook Me All Night Long')
(19, 'Chasing Pavements')
(20, "Don't You Remember")
(21, 'Easy On Me')
(22, "He Won't Go")
(23, 'Hello')
(24, 'Love In The Dark')
(25, 'Make You Feel My Love')
(26, 'Million Years Ago')
(27, 'One and Only')
(28, 'Remedy')
(29, 'Rolling in the Deep')
(30, 'Rumour Has It')
(31, 'Send My Love (To Your New Lover)')
(32, 'Set Fire to the Rain')
(33, 'Skyfall')
(34, 'Someone Like You')
(35, 'Sweetest Devotion')
(36, 'Take It All')
(37, 'Turning Tables')
(38, 'Water Under The Bridge')
(39, 'When We Were Young')
(40, 'A

In [1]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp", skip_duplicates=True)

  0%|          | 0/1053 [00:00<?, ?it/s]

  0%|          | 1/1053 [01:56<33:58:53, 116.29s/it]

  0%|          | 2/1053 [03:10<26:38:32, 91.26s/it] 

  0%|          | 3/1053 [03:14<15:07:26, 51.85s/it]

  0%|          | 4/1053 [07:29<38:27:33, 131.99s/it]

...

ERROR: [youtube] ocDlOD1Hw9k: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


[Attempt 1] Download error: ERROR: [youtube] ocDlOD1Hw9k: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


ERROR: [youtube] ocDlOD1Hw9k: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
 98%|█████████▊| 1034/1053 [15:09:17<31:17, 98.80s/it] 

[Attempt 2] Download error: ERROR: [youtube] ocDlOD1Hw9k: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Error indexing F**kin' Perfect (ID 737): cannot unpack non-iterable NoneType object


100%|██████████| 1053/1053 [15:40:18<00:00, 53.58s/it]   


In [1]:
from src.abracadabra.recognize import index_all_songs
index_all_songs(db_type="gcp", skip_duplicates=True, cookies=True)

Skipping Family Portrait (ID 738) — already indexed.
Skipping Funhouse (ID 739) — already indexed.
Skipping Funhouse - Main Version - Clean (ID 740) — already indexed.
Skipping Get the Party Started (ID 741) — already indexed.
Skipping Hustle (ID 742) — already indexed.
Skipping Just Like Fire (From the Original Motion Picture "Alice Through The Looking Glass") (ID 744) — already indexed.
Skipping Most Girls (ID 747) — already indexed.
Skipping Never Gonna Not Dance Again (ID 748) — already indexed.
Skipping Please Don't Leave Me (ID 749) — already indexed.
Skipping Raise Your Glass (ID 750) — already indexed.
Skipping Stupid Girls (ID 753) — already indexed.
Skipping TRUSTFALL (ID 754) — already indexed.
Skipping Today's The Day - From "The Ellen DeGeneres Show" (ID 756) — already indexed.
Skipping Trouble (ID 757) — already indexed.
Skipping Try (ID 758) — already indexed.
Skipping U + Ur Hand (ID 759) — already indexed.
Skipping Walk Me Home (ID 760) — already indexed.
Skipping What

  0%|          | 0/1053 [00:00<?, ?it/s]

100%|██████████| 1053/1053 [00:40<00:00, 25.86it/s]
